In [1]:
#Trivia game 
import requests
import json
from tkinter import *
import random 

root = Tk()
root.title("TV and Film Trivia")
root.geometry("500x550")

used_questions = []
question_count = 0
points_system = 0
correct_count = 0

def get_question():
    global used_questions, question_count
    if question_count == 20:
        q_button["state"] = "disabled"
        return
    text_box.delete("1.0", END)
    url = f"https://the-trivia-api.com/api/questions?categories=film_and_tv&limit=10&region=US&difficulty=easy"
    response = requests.get(url)
    data = json.loads(response.content.decode('utf-8')) #strings weren't reading/pulling right so this line of code fixed it https://stackoverflow.com/questions/13110629/decoding-utf-8-strings-in-python
    questions = data
    random_question = random.choice(questions)
    while random_question["question"] in used_questions:
        random_question = random.choice(questions) #randomizes the questions from the API
    used_questions.append(random_question["question"])
    question_count += 1
    
    #to begin to add answer choices in the same textbox as the question a new frame had to be created
    radio_frame = Frame(text_box, bg="light blue")
    text_box.window_create("end", window=radio_frame)
    text_box.insert("end", "\n\n") #to make the answers go down like a list and question to be in the same text box this line of code formatted it correctly https://www.tutorialspoint.com/printing-a-list-to-a-tkinter-text-widget
    
    
    question_label = Label(radio_frame, text=random_question["question"], bg="light blue", wraplength=450)
    question_label.pack(anchor="w")
    
    #putting in the actual answers as label buttons
    answer_options = get_answers(data, random_question["question"])
    random.shuffle(answer_options)
    for answer in answer_options:
        a_button = Radiobutton(radio_frame, text=answer, variable=user_answer, value=answer, bg="light blue")
        a_button.pack(anchor="w") #anchoring them west put them in the proper spot
    
    
    
    c_button["state"] = "normal" #allows the button to only be pressed at proper times, un"disables" the button https://stackoverflow.com/questions/16046743/how-to-change-tkinter-button-state-from-disabled-to-normal
    global correct_answer_options
    correct_answer_options = answer_options
    
   
    
def get_answers(response, question):
    answer_list = []
    global points_system
    for i in response:
        if i["question"] == question:
            cor = i["correctAnswer"]
            answer_list.append(cor)
            points_system += 1
            for incor in i["incorrectAnswers"]:
                answer_list.append(incor)
    return answer_list
    
def check_answer():
    global correct_count
    cor_ans = correct_answer_options[0]
    if user_answer.get() == cor_ans:
        feedback["text"] = "Correct :)"
        correct_count += 1
    else:
        feedback["text"] = "Wrong, please try again"
    
    #display label at the end showing how many questions the user got correct
    answer_label["text"] = f"Correct: {correct_count} / 20"
    

user_answer = StringVar()
user_answer.set("")
text_box = Text(root, bg= "light blue", height = 15, width = 100, padx = 10, pady = 10)
q_button = Button(root, text= "Click to get a question", command=get_question)
c_button = Button(root, text= "Check Answer", command=check_answer, state="disabled")
feedback = Label(root, text="", bg="light blue")
answer_label = Label(root, text="Correct: 0 / 20", bg="light blue")

text_box.pack()
q_button.pack()
c_button.pack()
feedback.pack()
answer_label.pack()

root.mainloop()
#for the most part I was able to figure out everything besides how to make the program more accurate. Sometimes, when a user guesses an answer that is right, the program will say it is wrong. After many different attempts and new/deleted lines of code, and checked websites, this is the closest I have gotten to accuracy.